In [337]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import mss
import time
import os
import warnings
import logging
from concurrent.futures import ThreadPoolExecutor

In [338]:
# CONSTANTS

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
VID_OUTPUT_DIR = 'video_out'        # Directory to save video output
LABEL = False                       # Whether or not to label during capture process      
FPS = 1                             # Frames to capture per second
SECONDS_TO_CAPTURE = 60             # Video duration

In [339]:
if DEVICE == torch.device('cuda'):
    print(f'Using {DEVICE} {torch.cuda.get_device_name()} with {torch.cuda.mem_get_info()[0]/(1024**2)} MB of VRAM')

Using cuda NVIDIA GeForce RTX 3050 Ti Laptop GPU with 3090.4500007629395 MB of VRAM


In [340]:
if not os.path.exists('yolov5'):
  !git clone https://github.com/ultralytics/yolov5
  !pip install -r yolov5/requirements.txt
  

if not os.path.exists(VID_OUTPUT_DIR):
  os.makedirs(VID_OUTPUT_DIR)
  
warnings.simplefilter("ignore", FutureWarning)

In [341]:
with mss.mss() as sct:
  print(sct.monitors)

[{'left': 0, 'top': 0, 'width': 3840, 'height': 1080}, {'left': 1920, 'top': 0, 'width': 1920, 'height': 1080}, {'left': 0, 'top': 0, 'width': 1920, 'height': 1080}]


In [342]:
logging.getLogger('ultralytics').setLevel(logging.ERROR)

In [343]:
frames_to_capture = FPS * SECONDS_TO_CAPTURE
frames = []
# yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# yolov5.eval()

yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)
yolov5.eval()

def detect(img, model):
  img = model(img)
  
  return img.render()[0]

def capture(region):
  with mss.mss() as sct:
    return sct.grab(region)

monitor_1 = mss.mss().monitors[1]
print(f'Monitor: {monitor_1}')
t, l, w, h = monitor_1['top'], monitor_1['left'], monitor_1['width'], monitor_1['height']
region_1 = {'left': l+int(w * 0.338), 'top': t, 'width': w-int(w * 0.673), 'height': h}
region_2 = {'left': l+int(w * 0.67), 'top': t, 'width': w-int(w * 0.98), 'height': h-int(h * 0.33)}
# region_2 = {'left': l+int(l * 0.859), 'top': t, 'width': w-int(w * 0.820), 'height': h-int(h * 0.952)}

print(f'Capturing regions: {region_1} and {region_2}')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_1 = cv2.VideoWriter(f'{VID_OUTPUT_DIR}/output_1.mp4', fourcc, FPS, (region_1['width'], region_1['height']))
out_2 = cv2.VideoWriter(f'{VID_OUTPUT_DIR}/output_2.mp4', fourcc, FPS, (region_2['width'], region_2['height']))
model = yolov5

with ThreadPoolExecutor(max_workers=8) as executor:
  for i in range(frames_to_capture):
    time_start = time.time()
    thread_1 = executor.submit(capture, region_1)
    thread_2 = executor.submit(capture, region_2)
    
    img_1 = thread_1.result()
    img_2 = thread_2.result()
    
    if LABEL:
      # img_1 = cv2.resize(np_img_1, (int(np_img_1.shape[0] * 0.5), int(np_img_1.shape[1] * 0.5)))
      # img_2 = cv2.resize(np_img_2, (int(np_img_2.shape[0] * 0.5), int(np_img_2.shape[1] * 0.5)))

      thread_1 = executor.submit(detect, np.array(img_1), model)
      thread_2 = executor.submit(detect, np.array(img_2), model)

      img_1 = thread_1.result()
      img_2 = thread_2.result()
    
    out_1.write(cv2.cvtColor(np.array(img_1), cv2.COLOR_BGRA2BGR))
    out_2.write(cv2.cvtColor(np.array(img_2), cv2.COLOR_BGRA2BGR))
    
    if time_start + 1/FPS > time.time():
      time.sleep(time_start + 1/FPS - time.time())

out_1.release()
out_2.release()
print(f'Capture region 1 saved to {VID_OUTPUT_DIR}/output_1.mp4')
print(f'Capture region 2 saved to {VID_OUTPUT_DIR}/output_2.mp4')

Using cache found in C:\Users\tiany/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-19 Python-3.10.6 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 


Monitor: {'left': 1920, 'top': 0, 'width': 1920, 'height': 1080}
Capturing regions: {'left': 2568, 'top': 0, 'width': 628, 'height': 1080} and {'left': 3206, 'top': 0, 'width': 39, 'height': 724}
Capture region 1 saved to video_out/output_1.mp4
Capture region 2 saved to video_out/output_2.mp4
